In [29]:
%pylab inline
plt.style.use("bmh")
import pandas as pd
import numpy as np
import pathlib
import matplotlib as mpl
import torch
from torch import optim, device, cuda

from training import Estimator, retrieve_data
from Models import MQRNN

Populating the interactive namespace from numpy and matplotlib


## The model features - 9 calendar features

The night / day cycle varies between summer and winter (see graph below that illustrates that).

In the summer and winter, the daily cycle is changing.

Therefore add just a daily cycle and yearly cycle features are not enough to identify whole annual cycles.

For example, in the summer the consumption peak is in the afternoon and winter probably in the evening. Also, the difference between the peak and minimum of daily consumption is varying throughout the year.
The following variables together try to predict the full yearly sub-cycles.

**Full calendar features ($x_h,x_f$):**
1. yearly_cycle = np.sin(2 * np.pi * df.index.dayofyear / 366)

2. weekly_cycle = np.sin(2 * np.pi * df.index.dayofweek / 7)

3. daily_cycle = np.sin(2 * np.pi * df.index.hour / 24)

4. weekend = (df.index.dayofweek < 5).astype(float)

5. night = ((df.index.hour < 7) & (df.index.hour>21)).astype(float)

6. winter = ((df.index.month < 4) & (df.index.month > 10)).astype(float)

7. Holiday = ((df.index.is_year_end | df.index.is_year_start) | (( df.index.month==12) & (df.index.daysinmonth==25))).astype(float)

8. winter_daily_cycle = daily_cycle*winter
9. summer_daily_cycle = daily_cycle*(1-winter)


`calendar_features = ["yearly_cycle", "weekly_cycle", "daily_cycle", "weekend", "night", "winter", "Holiday", "summer_daily_cycle", "winter_daily_cycle"]`


It should be noted that adding all nine features helps the prediction accuracy mainly when using a small sample. However, when the models are heavily trained and include 100 samples (per househould), these features contribution is minor. However, we added them because they improve the model accuracy.

In [30]:
input_size = 1  # y
embed_size = 9  # x
hidden_size = 30  # for lstm: "both with a state dimension of 30"
context_size = 8  # for c_t/c_a
horizon = 24
quantiles = [0.01, 0.25, 0.5, 0.75, 0.99]
samples = 10
batch_size = 32
epochs = 1
print_every = 50

In [31]:
DATA_DIR = pathlib.Path("data")
MODEL_DIR = pathlib.Path("models")
GRAPH_DIR = pathlib.Path("graphs")

MODEL_DIR.mkdir(exist_ok=True)
GRAPH_DIR.mkdir(exist_ok=True)

### Model with all 9 calendar_features

In [32]:
train_loader, val_loader, test_loader = retrieve_data(samples=samples)
model = MQRNN(input_size, embed_size, hidden_size, context_size, horizon, quantiles)

optimizer = optim.Adam(model.parameters(), lr=0.01, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',patience=4, factor=0.1, verbose=True)

estimator = Estimator(batch_size=batch_size, quantiles=quantiles)
estimator.train(model, train_loader, val_loader, optimizer, scheduler, epochs=epochs)

2021-03-09 03:55:35.458044 Starting epoch 1 / 1
2021-03-09 03:56:36.880409 t = 50, loss = 0.0704
2021-03-09 03:57:30.339981 Got average loss of (0.06) last_lr = 0.01


In [33]:
model.eval()
loss = 0
for t, (enc_data, dec_data) in enumerate(test_loader):
  loss += estimator.calculate_loss(model, enc_data, dec_data).item()
loss = loss / (t+1)
print("\n","The test average loss:", round(loss, 4))
torch.save(model.state_dict(), pathlib.Path("models").joinpath("epoch_{0}_loss_{1}".format(30, round(loss, 4))))


 The test average loss: 0.0569


In [34]:
dataiter = iter(test_loader)
enc_data, dec_data = dataiter.next()
estimator.forcast(model, enc_data, dec_data, quantiles = quantiles, input_size = 1)

In [35]:
import matplotlib.image as mpimg 
import matplotlib.pyplot as plt 

img1=mpimg.imread( pathlib.Path(GRAPH_DIR).joinpath(str(1)+".png")) 
img2=mpimg.imread( pathlib.Path(GRAPH_DIR).joinpath(str(2)+".png")) 
img3=mpimg.imread( pathlib.Path(GRAPH_DIR).joinpath(str(3)+".png")) 
img4=mpimg.imread( pathlib.Path(GRAPH_DIR).joinpath(str(4)+".png")) 

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2,figsize=(14,8))
fig.tight_layout()
ax1.imshow(img1)
ax1.axis("off")
ax2.imshow(img2)
ax2.axis("off")
ax3.imshow(img3)
ax3.axis("off")
ax4.imshow(img4)
ax4.axis("off")

TypeError: Object does not appear to be a 8-bit string path or a Python file-like object

## The average consumption across all 370 households - a graphical summary

note: the average does not include zeros only

In [ ]:
eldata = pd.read_csv('data\LD2011_2014.txt',
                 parse_dates=[0],
                 delimiter=";",
                 decimal=",")
eldata.rename({"Unnamed: 0": "timestamp"}, axis=1, inplace=True)

eldata=eldata.resample('H', on='timestamp').mean().reset_index()

el_mean=eldata[eldata!=0].set_index("timestamp").mean(axis=1)


## An illustration of the winter-summer cycles




The following graph shows the average consumption over several years.
The colors are arbitrary, illustrating the summer - winter seasons.



In [ ]:
plt.figure(figsize=(16,8))
el_mean.loc["2011-1-1":"2011-4-19"].plot()
el_mean.loc["2011-4-20":"2011-11-19"].plot()
el_mean.loc["2011-11-20":"2012-4-19"].plot()
el_mean.loc["2012-4-20":"2012-11-19"].plot()

el_mean.loc["2012-11-20":"2013-4-19"].plot()
el_mean.loc["2013-4-20":"2013-11-19"].plot()

el_mean.loc["2013-11-20":"2014-4-19"].plot()
el_mean.loc["2014-4-20":"2014-11-19"].plot()
el_mean.loc["2014-11-20":"2015-1-31"].plot()


plt.tight_layout()

The minimum daily consumption has moderate yearly cycles, and the maximum has high cycles.